` Deliverable URLs of the colab notebook: `


https://colab.research.google.com/drive/17wYGMvMr0YnFYWwdgaMfHbsSsFWJjYGf?usp=sharing`

In [ ]:
!pip install pandas scikit-learn nltk

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [28]:
# Removing stop works and usernames
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import *
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TANISHQ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TANISHQ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# `Hate Detection`

In [29]:
train_hate = pd.read_csv('data/hate/train.csv')
val_hate = pd.read_csv('data/hate/val.csv')

In [30]:
train_hate.shape, val_hate.shape

((3660, 2), (457, 2))

In [130]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(pd.DataFrame(train_hate))

In [131]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

remove arabic words
dot(.) is used instead of space
can just remove punctuations
Remove twitter or any types of links (yt links also present)
lowercase everything
remove @usernames
remove numbers
same words used in many contexts both hate and non hate like rape, murder
hate word is used in many hate sentences
words that are pronuns instead of nouns are more directed to sarcasm. Like TU, tera rather than someones name - just random assumption not correct


In [132]:
hateCount_train = len(train_hate[train_hate['Tag'] == 1])
nothateCount_train = len(train_hate[train_hate['Tag'] == 0])
hateCount_val = len(val_hate[val_hate['Tag'] == 1])
nothateCount_val = len(val_hate[val_hate['Tag'] == 0])

word_count_df = pd.DataFrame(
         {
        'Number of words': ["Hate Words", "Not Hate Words"],
        'Training Set': [hateCount_train, nothateCount_train],
        'Validation Set': [hateCount_val, nothateCount_val]
        },
    )
word_count_df

,Number of words,Training Set,Validation Set
0,Hate Words,1353,148
1,Not Hate Words,2307,309


In [133]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))

# Fit and transform the text data
X_train = vectorizer.fit_transform(train_hate["Sentence"])
# Convert sparse matrix to array for better readability
# X_array = X.toarray()


In [33]:

class TextClassifier:
    def __init__(self, model_type="svm_linear"):
        """
        Initialize the classifier.

        model_type: 
            "svm_linear" -> Linear SVM (fast for high-dim features)
            "svm_rbf"    -> RBF SVM (slow for large data)
            "naive_bayes" -> Multinomial Naïve Bayes (best for text)
            "logistic_regression" -> Logistic Regression
        """
        self.model_type = model_type
        self.model = self._initialize_model(model_type)

    def _initialize_model(self, model_type):
        """Initialize the selected model."""
        if model_type == "svm_linear":
            return SVC(kernel="linear",verbose=True, gamma='auto')
        elif model_type == "svm_rbf":
            return SVC(kernel="rbf",  verbose=True)
        elif model_type == "naive_bayes":
            return MultinomialNB()
        elif model_type == "logistic_regression":
            return LogisticRegression(class_weight='balanced')
        elif model_type == "random_forest":
            return RandomForestClassifier(class_weight='balanced')
        else:
            raise ValueError("Invalid model_type. Choose from 'svm_linear', 'svm_rbf', 'naive_bayes', 'logistic_regression'.")

    def fit(self, X_train, y_train):
        """Train the model."""
        print(f"Training {self.model_type} model...")
        self.model.fit(X_train, y_train)
        print("Training completed.")

    def predict(self, X_test):
        """Predict labels for new data."""
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        """Evaluate the model using accuracy, precision, recall, and F1-score."""
        y_pred = self.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        matrix = confusion_matrix(y_test, y_pred)

        print(f"\n🔹 Model: {self.model_type}")
        print(f"✅ Accuracy: {accuracy:.4f}")
        print(report)
        print(matrix)
        
    def predict_proba(self, X_test):
        """Predict probabilities for each class."""
        return self.model.predict_proba(X_test)
    


## `SVM Linear Classifier`

In [135]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_linear")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])



Training svm_linear model...
[LibSVM]Training completed.


In [136]:
X_test = vectorizer.transform(val_hate["Sentence"])
y_test = val_hate['Tag']

# Evaluate on test data
classifier.evaluate(X_test, y_test)
# y_pred = sgd_svm.predict(X_test)



🔹 Model: svm_linear
✅ Accuracy: 0.7002
              precision    recall  f1-score   support

           0       0.74      0.86      0.79       309
           1       0.56      0.37      0.45       148

    accuracy                           0.70       457
   macro avg       0.65      0.61      0.62       457
weighted avg       0.68      0.70      0.68       457

[[265  44]
 [ 93  55]]


## `RBF Kernel SVM`

In [137]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_rbf")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])



Training svm_rbf model...
[LibSVM]Training completed.


In [138]:
X_test = vectorizer.transform(val_hate["Sentence"])
y_test = val_hate['Tag']

# Evaluate on test data
classifier.evaluate(X_test, y_test)
# y_pred = sgd_svm.predict(X_test)


🔹 Model: svm_rbf
✅ Accuracy: 0.7002
              precision    recall  f1-score   support

           0       0.70      0.98      0.82       309
           1       0.76      0.11      0.19       148

    accuracy                           0.70       457
   macro avg       0.73      0.55      0.50       457
weighted avg       0.72      0.70      0.61       457

[[304   5]
 [132  16]]


## `Naive Bayes`

In [139]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="naive_bayes")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])

Training naive_bayes model...
Training completed.


In [140]:
X_test = vectorizer.transform(val_hate["Sentence"])
y_test = val_hate['Tag']

# Evaluate on test data
classifier.evaluate(X_test.toarray(), y_test)


🔹 Model: naive_bayes
✅ Accuracy: 0.7177
              precision    recall  f1-score   support

           0       0.72      0.95      0.82       309
           1       0.69      0.24      0.35       148

    accuracy                           0.72       457
   macro avg       0.70      0.59      0.59       457
weighted avg       0.71      0.72      0.67       457

[[293  16]
 [113  35]]


## `Logistic Regression`

In [141]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="logistic_regression")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])

Training logistic_regression model...
Training completed.


In [142]:
X_test = vectorizer.transform(val_hate["Sentence"])
y_test = val_hate['Tag']

# Evaluate on test data
classifier.evaluate(X_test.toarray(), y_test)


🔹 Model: logistic_regression
✅ Accuracy: 0.6805
              precision    recall  f1-score   support

           0       0.73      0.83      0.78       309
           1       0.51      0.36      0.42       148

    accuracy                           0.68       457
   macro avg       0.62      0.60      0.60       457
weighted avg       0.66      0.68      0.66       457

[[258  51]
 [ 95  53]]


## `Random Forest`

In [143]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="random_forest")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])

Training random_forest model...
Training completed.


In [144]:
X_test = vectorizer.transform(val_hate["Sentence"])
y_test = val_hate['Tag']

# Evaluate on test data
classifier.evaluate(X_test.toarray(), y_test)


🔹 Model: random_forest
✅ Accuracy: 0.7046
              precision    recall  f1-score   support

           0       0.71      0.97      0.82       309
           1       0.70      0.16      0.25       148

    accuracy                           0.70       457
   macro avg       0.70      0.56      0.53       457
weighted avg       0.70      0.70      0.63       457

[[299  10]
 [125  23]]


In [145]:
# Load custom stopwords from a text file
def load_stopwords(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        stopwords = set(file.read().splitlines())  # Read line by line into a set
    return stopwords

# Example usage
custom_stopwords = load_stopwords("stop_hinglish.txt")


## `Preprocessing words`

In [ ]:


def preprocess_text(text):
    # # # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
   
    # removing usernames from text would lead to loss of info as hate is directed after mentions 
    # text = re.sub(r"@[A-Za-z0-9]+", "", text)

    text = re.sub("[^a-zA-Z]", " ", text)
    # # Remove stopwords
    # stop_words = set(stopwords.words('english'))
    # text = ' '.join([word for word in text.split() if word not in stop_words])


    
    return text

In [202]:
# Runinng the model with preprocessed text
X_train = train_hate['Sentence'].apply(preprocess_text)
X_val = val_hate['Sentence'].apply(preprocess_text)

# train_hate['Sentence']

In [203]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))

# Fit and transform the text data
X_train = vectorizer.fit_transform(X_train)


In [204]:
classifier = TextClassifier(model_type="naive_bayes")

# Train on dataset
classifier.fit(X_train, train_hate['Tag'])




Training naive_bayes model...
Training completed.


In [205]:

X_test = vectorizer.transform(val_hate["Sentence"])

y_test = val_hate['Tag']



In [206]:
classifier.evaluate(X_test, y_test)



🔹 Model: naive_bayes
✅ Accuracy: 0.7155
              precision    recall  f1-score   support

           0       0.72      0.95      0.82       309
           1       0.68      0.23      0.34       148

    accuracy                           0.72       457
   macro avg       0.70      0.59      0.58       457
weighted avg       0.71      0.72      0.66       457

[[293  16]
 [114  34]]


# `Humour Classification`

In [238]:
train_humor = pd.read_csv('data/humor/train.csv')
val_humor = pd.read_csv('data/humor/val.csv')

In [239]:
train_humor

,Sentence,Tag
0,"Jyotiraditya Scindia is like ""Rassi jal gayee ...",1
1,Ishant Sharma ko bahut late utaara.,1
2,.@twinitisha neeche plug nikla hua hai..,0
3,Aaj agar India final me hota to kam se kam New...,0
4,3 stages of life of Mechanical Engineer:\n\n1)...,1
...,...,...
2355,Nitish & Lalu scared of Modi so much that agar...,1
2356,.@ShirishKunder Sabzi bana ke rakhna nahi to F...,1
2357,"""BARKHA se bacha lu tujhe seene se laga lu, aa...",0
2358,"""Bol raha hai mere saath debate karega.. Muh k...",0


In [242]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))

# Fit and transform the text data
X_train_humor = vectorizer.fit_transform(train_humor["Sentence"])


## `SVM Linear Classifier`

In [243]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_linear")

# Train on dataset
classifier.fit(X_train_humor, train_humor['Tag'])



Training svm_linear model...
[LibSVM]Training completed.


In [244]:

X_test_humor = vectorizer.transform(val_humor["Sentence"])

y_test_humor = val_humor['Tag']

# Evaluate on test data

In [245]:
classifier.evaluate(X_test_humor, y_test_humor)



🔹 Model: svm_linear
✅ Accuracy: 0.7051
              precision    recall  f1-score   support

           0       0.62      0.67      0.65       119
           1       0.77      0.73      0.75       176

    accuracy                           0.71       295
   macro avg       0.70      0.70      0.70       295
weighted avg       0.71      0.71      0.71       295

[[ 80  39]
 [ 48 128]]


## `SVM RBF Classifier`

In [246]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_rbf")

# Train on dataset
classifier.fit(X_train_humor, train_humor['Tag'])



Training svm_rbf model...
[LibSVM]Training completed.


In [247]:

X_test_humor = vectorizer.transform(val_humor["Sentence"])

y_test_humor = val_humor['Tag']
classifier.evaluate(X_test_humor, y_test_humor)



🔹 Model: svm_rbf
✅ Accuracy: 0.4949
              precision    recall  f1-score   support

           0       0.44      0.92      0.60       119
           1       0.80      0.20      0.33       176

    accuracy                           0.49       295
   macro avg       0.62      0.56      0.46       295
weighted avg       0.65      0.49      0.43       295

[[110   9]
 [140  36]]


## `Naive Bayes Classifier`

In [248]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="naive_bayes")

# Train on dataset
classifier.fit(X_train_humor, train_humor['Tag'])



Training naive_bayes model...
Training completed.


In [249]:

X_test_humor = vectorizer.transform(val_humor["Sentence"])

y_test_humor = val_humor['Tag']
classifier.evaluate(X_test_humor, y_test_humor)



🔹 Model: naive_bayes
✅ Accuracy: 0.6542
              precision    recall  f1-score   support

           0       0.71      0.24      0.36       119
           1       0.65      0.93      0.76       176

    accuracy                           0.65       295
   macro avg       0.68      0.59      0.56       295
weighted avg       0.67      0.65      0.60       295

[[ 29  90]
 [ 12 164]]


## `Logistic Regression`

In [250]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="logistic_regression")

# Train on dataset
classifier.fit(X_train_humor, train_humor['Tag'])



Training logistic_regression model...
Training completed.


In [251]:

X_test_humor = vectorizer.transform(val_humor["Sentence"])

y_test_humor = val_humor['Tag']
classifier.evaluate(X_test_humor, y_test_humor)



🔹 Model: logistic_regression
✅ Accuracy: 0.6847
              precision    recall  f1-score   support

           0       0.60      0.63      0.62       119
           1       0.74      0.72      0.73       176

    accuracy                           0.68       295
   macro avg       0.67      0.68      0.67       295
weighted avg       0.69      0.68      0.69       295

[[ 75  44]
 [ 49 127]]


## `Random Forest`

In [252]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="random_forest")

# Train on dataset
classifier.fit(X_train_humor, train_humor['Tag'])



Training random_forest model...
Training completed.


In [253]:

X_test_humor = vectorizer.transform(val_humor["Sentence"])

y_test_humor = val_humor['Tag']
classifier.evaluate(X_test_humor, y_test_humor)



🔹 Model: random_forest
✅ Accuracy: 0.6475
              precision    recall  f1-score   support

           0       0.55      0.69      0.61       119
           1       0.75      0.62      0.68       176

    accuracy                           0.65       295
   macro avg       0.65      0.65      0.64       295
weighted avg       0.67      0.65      0.65       295

[[ 82  37]
 [ 67 109]]


In [146]:
train_humor = pd.read_csv('data/humor/train.csv')
val_humor = pd.read_csv('data/humor/val.csv')

In [147]:
train_humor

,Sentence,Tag
0,"Jyotiraditya Scindia is like ""Rassi jal gayee ...",1
1,Ishant Sharma ko bahut late utaara.,1
2,.@twinitisha neeche plug nikla hua hai..,0
3,Aaj agar India final me hota to kam se kam New...,0
4,3 stages of life of Mechanical Engineer:\n\n1)...,1
...,...,...
2355,Nitish & Lalu scared of Modi so much that agar...,1
2356,.@ShirishKunder Sabzi bana ke rakhna nahi to F...,1
2357,"""BARKHA se bacha lu tujhe seene se laga lu, aa...",0
2358,"""Bol raha hai mere saath debate karega.. Muh k...",0


In [ ]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))

# Fit and transform the text data
X_train_humor = vectorizer.fit_transform(train_humor["Sentence"])


## `Preprocessing words`

In [352]:

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Removing words that start with numbers (e.g., usernames like "123abc")
    text = re.sub(r"\b\d+\w*", "", text)
    
    # Replace dots with space
    text = text.replace(".", " ")
    
    # Remove punctuation and special characters (keeping only alphabets and spaces)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    # Tokenization (Splitting into words)
    words = text.split()

    # Reconstruct text
    text = " ".join(words)
    
    return text


In [353]:
# Runinng the model with preprocessed text
X_train = train_humor['Sentence'].apply(preprocess_text)
X_val = val_humor['Sentence'].apply(preprocess_text)

# train_hate['Sentence']

In [354]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))

# Fit and transform the text data
X_train = vectorizer.fit_transform(X_train)


In [355]:
classifier = TextClassifier(model_type="svm_linear")

# Train on dataset
classifier.fit(X_train, train_humor['Tag'])




Training svm_linear model...
[LibSVM]Training completed.


In [356]:

X_test = vectorizer.transform(val_humor["Sentence"])

y_test = val_humor['Tag']



In [357]:
classifier.evaluate(X_test, y_test)



🔹 Model: svm_linear
✅ Accuracy: 0.7186
              precision    recall  f1-score   support

           0       0.64      0.70      0.67       119
           1       0.78      0.73      0.76       176

    accuracy                           0.72       295
   macro avg       0.71      0.72      0.71       295
weighted avg       0.72      0.72      0.72       295

[[ 83  36]
 [ 47 129]]


# `Sarcasm Classification`

In [358]:
train_sarcasm = pd.read_csv('data/sarcasm/train.csv')
val_sarcasm = pd.read_csv('data/sarcasm/val.csv')

In [359]:
sarcasmCount_train = len(train_sarcasm[train_sarcasm['Tag'] == 1])
notsarcasmCount_train = len(train_sarcasm[train_sarcasm['Tag'] == 0])
sarcasmCount_val = len(val_sarcasm[val_sarcasm['Tag'] == 1])
notsarcasmCount_val = len(val_sarcasm[val_sarcasm['Tag'] == 0])

word_count_df = pd.DataFrame(
         {
        'Number of words': ["Sarcasm Words", "Not Sarcasm Words"],
        'Training Set': [sarcasmCount_train, notsarcasmCount_train],
        'Validation Set': [sarcasmCount_val, notsarcasmCount_val]
        },
    )
word_count_df

,Number of words,Training Set,Validation Set
0,Sarcasm Words,403,51
1,Not Sarcasm Words,3797,474


## `SVM Linear Classifier`

In [361]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))

# Fit and transform the text data
X_train_sarcasm = vectorizer.fit_transform(train_sarcasm["Sentence"])
# Convert sparse matrix to array for better readability
# X_array = X.toarray()


In [362]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_linear")

# Train on dataset
classifier.fit(X_train_sarcasm, train_sarcasm['Tag'])



Training svm_linear model...
[LibSVM]Training completed.


In [363]:

X_test_sarcasm = vectorizer.transform(val_sarcasm["Sentence"])

y_test_sarcasm = val_sarcasm['Tag']

# Evaluate on test data

In [364]:
classifier.evaluate(X_test_sarcasm, y_test_sarcasm)



🔹 Model: svm_linear
✅ Accuracy: 0.9676
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       474
           1       0.90      0.75      0.82        51

    accuracy                           0.97       525
   macro avg       0.94      0.87      0.90       525
weighted avg       0.97      0.97      0.97       525

[[470   4]
 [ 13  38]]


## `SVM RBF Classifier`

In [365]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="svm_rbf")

# Train on dataset
classifier.fit(X_train_sarcasm, train_sarcasm['Tag'])



Training svm_rbf model...
[LibSVM]Training completed.


In [366]:

X_test_sarcasm = vectorizer.transform(val_sarcasm["Sentence"])

y_test_sarcasm = val_sarcasm['Tag']
classifier.evaluate(X_test_sarcasm, y_test_sarcasm)



🔹 Model: svm_rbf
✅ Accuracy: 0.9562
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       474
           1       0.83      0.69      0.75        51

    accuracy                           0.96       525
   macro avg       0.90      0.84      0.86       525
weighted avg       0.95      0.96      0.95       525

[[467   7]
 [ 16  35]]


## `Naive Bayes Classifier`

In [232]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="naive_bayes")

# Train on dataset
classifier.fit(X_train_sarcasm, train_sarcasm['Tag'])



Training naive_bayes model...
Training completed.


In [233]:

X_test_sarcasm = vectorizer.transform(val_sarcasm["Sentence"])

y_test_sarcasm = val_sarcasm['Tag']
classifier.evaluate(X_test_sarcasm, y_test_sarcasm)



🔹 Model: naive_bayes
✅ Accuracy: 0.9048
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       474
           1       1.00      0.02      0.04        51

    accuracy                           0.90       525
   macro avg       0.95      0.51      0.49       525
weighted avg       0.91      0.90      0.86       525

[[474   0]
 [ 50   1]]


## `Logistic Regression`

In [234]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="logistic_regression")

# Train on dataset
classifier.fit(X_train_sarcasm, train_sarcasm['Tag'])



Training logistic_regression model...
Training completed.


In [235]:

X_test_sarcasm = vectorizer.transform(val_sarcasm["Sentence"])

y_test_sarcasm = val_sarcasm['Tag']
classifier.evaluate(X_test_sarcasm, y_test_sarcasm)



🔹 Model: logistic_regression
✅ Accuracy: 0.9619
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       474
           1       0.76      0.88      0.82        51

    accuracy                           0.96       525
   macro avg       0.87      0.93      0.90       525
weighted avg       0.97      0.96      0.96       525

[[460  14]
 [  6  45]]


## `Random Forest`

In [236]:
# Initialize with SVM (linear)
classifier = TextClassifier(model_type="random_forest")

# Train on dataset
classifier.fit(X_train_sarcasm, train_sarcasm['Tag'])



Training random_forest model...
Training completed.


In [237]:

X_test_sarcasm = vectorizer.transform(val_sarcasm["Sentence"])

y_test_sarcasm = val_sarcasm['Tag']
classifier.evaluate(X_test_sarcasm, y_test_sarcasm)



🔹 Model: random_forest
✅ Accuracy: 0.9067
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       474
           1       1.00      0.04      0.08        51

    accuracy                           0.91       525
   macro avg       0.95      0.52      0.51       525
weighted avg       0.92      0.91      0.87       525

[[474   0]
 [ 49   2]]


## `Preprocessing words`

In [ ]:

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    
    # Removing usernames (@mentions can be useful for context, so it's optional)
    text = re.sub(r"@[A-Za-z0-9]+", "", text)
    
    # Removing words that start with numbers (e.g., usernames like "123abc")
    text = re.sub(r"\b\d+\w*", "", text)
    
    # Replace dots with space
    text = text.replace(".", " ")
    
    # Remove punctuation and special characters (keeping only alphabets and spaces)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    
    return text


In [388]:
# Runinng the model with preprocessed text
X_train = train_sarcasm['Sentence'].apply(preprocess_text)
X_val = val_sarcasm['Sentence'].apply(preprocess_text)

# train_hate['Sentence']

In [389]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))

# Fit and transform the text data
X_train = vectorizer.fit_transform(X_train)


In [390]:
classifier = TextClassifier(model_type="svm_linear")

# Train on dataset
classifier.fit(X_train, train_sarcasm['Tag'])




Training svm_linear model...
[LibSVM]Training completed.


In [391]:

X_test = vectorizer.transform(val_sarcasm["Sentence"])

y_test = val_sarcasm['Tag']



In [392]:
classifier.evaluate(X_test, y_test)



🔹 Model: svm_linear
✅ Accuracy: 0.9657
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       474
           1       0.92      0.71      0.80        51

    accuracy                           0.97       525
   macro avg       0.95      0.85      0.89       525
weighted avg       0.96      0.97      0.96       525

[[471   3]
 [ 15  36]]
